<a href="https://colab.research.google.com/github/jhyglory/Section1/blob/main/section2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1) 데이터 선정 이유 및 문제 정의**




지난 1년간 우리 회사의 보험계약자가 우리 회사가 제공하는 자동차 보험에 관심이 있을지를 예측 해보고자 합니다.
고객이 자동차 보험에 관심을 가질지 여부를 예측하는 것은 그에 따라 고객에게 비즈니스 모델과 수익을 최적화하기 위한 커뮤니케이션 전략을 계획할 수 있기 때문에 우리 회사에 매우 도움이 될 것입니다.

가입의사가 있는지 없는지 예측하는 것이므로 분류문제(1,0)로 풀겠습니다.

In [ ]:
!pip install pdpbox

In [ ]:
!pip install shap

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# target: Respones

In [ ]:
import pandas as pd
train = pd.read_csv('/gdrive/MyDrive/project/train.csv')
test = pd.read_csv('/gdrive/MyDrive/project/test.csv')
train

In [ ]:
test.head()


* id :  고객의 고유 ID (Unique ID for the customer)

* Gender : 고객의 성별 (Gender of the customer)

* Age : 고객의 나이 (Age of the customer)

* Driving_License : 0 - 운전면허가 없는 고객 (Customer does not have DL), 1 - 운전면하가 있는 고객 (Customer already has DL)

* Region_Code : 고객 지역의 고유 코드 (Unique code for the region of the customer)

* Previously_Insured : 자동차 보험 가입 여부, 1 - 이미 자동차 보험에 가입 한 고객 (Customer already has Vehicle Insurance) , 0 - 자동차 보험에 가입하지 않은 고객 (Customer doesn't have Vehicle Insurance)

* Vehicle_Age : 차량 연식 (Age of the Vehicle) 

* Vehicle_Damage : 차량 데미지, 1 - 고객이 과거에 차량에 손상을 입힘. (Customer got his/her vehicle damaged in the past.) 0 - 고객이 과거에 차량에 손상시키지 않음. (Customer didn't get his/her vehicle damaged in the past.)

* Annual_Premium : 고객이 연도에 보험료로 지불해야 하는 금액 (The amount customer needs to pay as premium in the year)

* PolicySalesChannel : 고객에게 연락하는 채널을 위한 익명화 된 코드 (Anonymized Code for the channel of outreaching to the customer ie.) Different Agents, Over Mail, Over Phone, In Person, etc.

* Vintage : 일수 (Number of Days), Customer has been associated with the company

* Response : 응답, 1 - 자동차 보험에 관심 있음 (Customer is interested), 0 - 자동차 보험에 관심없음 (Customer is not interested)

2) 데이터를 이용한 가설 및 평가지표, 베이스라인 선택

가설: Vehicle_Age(차량 연식) 와 Vehicle_Damage(차량 데미지)가 자동차 보험 관심에 영향을 줄 것이라고 생각됩니다.

사용할 평가지표:
* Accuracy: 모든 분류 건수 중에서 분류기가 몇개의 정답을 맞혔는가 (맞거나 틀리다고 정확히 분류했는가)
* Recall: 맞다고 분류해야 하는 건수 중에서 분류기가 몇개나 제대로 분류했는가
* Precision: 분류기가 맞다고 분류한 건수 중에서 실제로 맞는 건수가 몇개나 되는가
 

* F1 score: Recall과 Precision의 균형값 (조화평균)




In [ ]:
# train 데이터 확인, 결측치 확인
print(train.info())
print(train.isnull().sum())

In [ ]:
# test 데이터 확인, 결측치 확인
print(test.info())
print(test.isnull().sum())

train 데이터에는 381109개의 행이 있고 test 데이터에는 127037개의 행이 있습니다.
결측치는 없습니다.

In [ ]:
# train 데이터 통계 요약
train.describe()

In [ ]:
target = 'Response'
y_train = train[target]
y_train.value_counts(normalize=True)

In [ ]:
# 'Response' 클래스 비율
import seaborn as sns
sns.set_theme(style='whitegrid')
sns.countplot(x=y_train)

자동차 보험에 관심을 보이는 고객이 12%로 현저히 낮습니다.
->클래스가 불균형

In [ ]:
train.groupby(['Response','Vehicle_Age','Vehicle_Damage']).size()

자동차 보험에 관심을 보이는 고객의 차량은 대부분 1-2년 되었고 차량이 손상된 적이 있습니다.

In [ ]:
# mode(): Return the highest frequency value in a Series.
major = y_train.mode()[0]

# 기준모델로 예측
y_pred = [major] * len(y_train)

In [ ]:
from sklearn.metrics import accuracy_score
print("training accuracy: ", accuracy_score(y_train, y_pred))

In [ ]:
# 'Gender' 클래스 비율, 성별에 따른 자동차 보험 관심 여부
import matplotlib.pyplot as plt
plt.figure(figsize = (15,10))
plt.subplots_adjust(left=0.5, bottom=0.5, right=0.9, top=0.9, wspace=0.7, hspace=0.5)
plt.subplot(1,2,1)
sns.countplot(train['Gender'],palette='Set2')
plt.title("count of male and female")
plt.subplot(1,2,2)
sns.countplot(train['Gender'], hue = train['Response'],palette="Set2")
plt.title("Response in Male and female category")
plt.show()

In [ ]:
# 연령에 따른 자동차 보험 관심 여부
plt.figure(figsize=(20,10))
sns.countplot(x='Age',hue='Response',data=train)

20대는 자동차 보험에 관심이 없습니다. 아직 많은 경험이 없고 비싼 차량이 없기 때문이라고 생각합니다. 30세-55세 사이의 사람들이 관심을 가장 많이 갖고 있습니다.

In [ ]:
# 'Age' 이상치 확인
sns.boxplot(train['Age'])

'Age'에 이상치가 없음을 알 수 있습니다.

In [ ]:
# 운전 면허 보유 여부에 따른 자동차 보험 관심 여부
sns.countplot(train['Driving_License'],hue=train['Response'])

In [ ]:
# 자동차 보험 가입 여부에 따른 자동차 보험 관심 여부
sns.countplot(x='Previously_Insured',hue='Response',data=train, palette='Accent')

이미 자동차 보험에 가입한 고객은 자동차 보험에 관심이 없습니다. 아직 이전 보험 계약이 만료되지 않았기 때문이라고 추측할 수 있습니다.

In [ ]:
# 차량 연식에 따른 자동차 보험 관심 여부
sns.countplot(x='Vehicle_Age',hue='Response',data=train, palette='Accent')

차량 연식이 1-2년인 경우가 가장 자동차 보험에 관심을 보였고 차량 연식이 1년 이하인 경우가 가장 관심이 낮았습니다.

In [ ]:
#Label Encoder
train['Gender'] = train['Gender'].replace({'Male':1,'Female':0})
train['Vehicle_Age'] = train['Vehicle_Age'].replace({'< 1 Year':0,'1-2 Year':1,'> 2 Years':2})
train['Vehicle_Damage'] = train['Vehicle_Damage'].replace({'Yes':1,'No':0})

test['Gender'] = test['Gender'].replace({'Male':1,'Female':0})
test['Vehicle_Age'] = test['Vehicle_Age'].replace({'< 1 Year':0,'1-2 Year':1,'> 2 Years':2})
test['Vehicle_Damage'] = test['Vehicle_Damage'].replace({'Yes':1,'No':0})

Gender
* 'Female' = 0
* 'Male' = 1

Vehicle Age
* '<1 Year' = 0
* '1–2 Year' = 1  
* '>2 Year' = 2

Vehicle Damage
* 'No' = 0
* 'Yes' = 1

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(train.corr(),annot=True, linewidths=2, cmap='YlGnBu')

* 자동차 보험 관심에 가장 영향을 미치는 요인은 양의 상관관계로 'Vehicle_Damage(차량 손상 정도)', 그 다음이 'Vehicle_Age(차량 연식)' 이고 음의 상관관계는 'Previously_Insured(자동차 보험 가입 여부)'이고 그 다음이 'Policy_Sales_Channel(고객에게 연락하는 채널 코드)'입니다. 

* 'Vintage'가 가장 영향을 적게 받습니다. 상관관계가 낮으므로 제거할 수 있습니다.

In [ ]:
#중복된 특성 확인
duplicate=train[train.duplicated()]
print(duplicate)

중복된 특성이 없는 것으로 확인됩니다.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import plot_confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from xgboost import XGBClassifier

In [ ]:
# 학습에 기여하지 않을 id 열 삭제
train = train.drop(['id', 'Vintage'], axis = 1)

In [ ]:
X = train[train.columns[:-1]]
y = train[train.columns[-1]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 30)

In [ ]:
RF_C= RandomForestClassifier(n_estimators = 30, 
                           max_depth=7,
                           n_jobs=-1,
                           class_weight='balanced', 
                           random_state=2)
RF_C.fit(X_train,y_train)
y_pred = RF_C.predict(X_test)

print(roc_auc_score(y_pred,y_test))

In [ ]:
print(classification_report(y_pred,y_test))

* accuracy: 정확도. 전체 학습데이터의 개수에서 각 클래스에서 자신의 클래스를 정확하게 맞춘 개수의 비율.

* macro arv: 단순평균

* weighted arv: 각 클래스에 속하는 표본의 갯수로 가중평균


In [ ]:
boosting=XGBClassifier(n_estimators = 1000, max_depth = 10, learning_rate = 0.2, n_jobs = -1 )
XGB = boosting.fit(X_train, y_train)
y_predict = XGB.predict(X_test)
xgb_probability = XGB.predict_proba(X_test)[:,1]
early_stopping_rounds = 50

acc_xgb=accuracy_score(y_test,y_predict)
recall_xgb=recall_score(y_test,y_predict)
precision_xgb=precision_score(y_test,y_predict)
f1score_xgb=f1_score(y_test,y_predict)

print(roc_auc_score(y_pred,y_test))

In [ ]:
print(classification_report(y_predict,y_test))

In [ ]:
import matplotlib.pyplot as plt
from pdpbox.pdp import pdp_isolate, pdp_plot

isolated = pdp_isolate(
    model = boosting,
    dataset = X_test, 
    model_features=X_test.columns, 
    feature = 'Vehicle_Damage',
    grid_type='percentile', # default='percentile', or 'equal'
    num_grid_points = 10 # default=10
)
pdp_plot(isolated, feature_name = feature);

차량 손상이 있을 때(1) 타겟변수에 대한 확률이 올라갑니다.

In [ ]:
isolated = pdp_isolate(
    model = boosting,
    dataset = X_test, 
    model_features=X_test.columns, 
    feature = 'Vehicle_Age',
    grid_type='percentile', # default='percentile', or 'equal'
    num_grid_points = 10 # default=10
)
pdp_plot(isolated, feature_name = feature);

Vehicle_Age가 2년 이상일 때 차 보험에 대한 긍정적인 의사 확률이 높아지는 것을 알 수 있습니다.

In [ ]:
from pdpbox.pdp import pdp_interact, pdp_interact_plot
features = ['Vehicle_Damage', 'Vehicle_Age']

interaction = pdp_interact(
    model=boosting, 
    dataset=X_test,
    model_features=X_test.columns, 
    features=features
)
pdp_interact_plot(interaction, plot_type='grid', 
                  feature_names=features);

두 가지 특성을 함께 봤을 때에는 Vehicle_Age의 정도차이는 없고, Vehicle_Damage의 차이가 타겟변수에 영향을 미친다는 것을 확인할 수 있습니다. 연식이 오래되지 않은 차 이면서 차량 손상이 있는 경우 가장 영향을 많이 미칩니다.

In [ ]:
import shap
shap_values = explainer.shap_values(X_test.iloc[:100])
shap.force_plot(explainer.expected_value, shap_values, X_test.iloc[:100])
shap.initjs()
shap_values = explainer.shap_values(X_test.iloc[:300])
shap.summary_plot(shap_values, X_train.iloc[:300])

In [ ]:
shap.summary_plot(shap_values, X_train.iloc[:300], plot_type="violin")

In [ ]:
shap.summary_plot(shap_values, X_train.iloc[:300], plot_type="bar")